In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torchvision import utils
import torchvision.transforms as T
import torch.utils.data as Data
from PIL import Image
import numpy as np
import torch.optim as optim
import cv2
import matplotlib.pyplot as plt





class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.Conv1 = nn.Sequential(
            #Convolution Level 1
            nn.Conv2d(1, 16, 5, 1, 2),
            #Activation function layer
            nn.ReLU(),
            #Maximum pooled layer
            nn.MaxPool2d(kernel_size = 2)
        )
        self.Conv2 = nn.Sequential(
            #Convolution Layer 2
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.Dropout(p=0.2),
            #Activation function layer
            nn.ReLU(),
            #Maximum pooled layer
            nn.MaxPool2d(kernel_size = 2)
        )
        #Finally, attach a fully connected layer (to make the image one-dimensional)
        #Why 32*7*7:(1,28,28) -> (16,28,28) (conv1) -> (16,14,14) (pool1) -> (32,14,14) (conv2) -> (32,7,7) (pool2) ->output
        self.Linear = nn.Sequential(
            nn.Linear(32*7*7,400),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(400,80),
            nn.ReLU(),
            nn.Linear(80,26),
         )

    def forward(self, input):
        input = self.Conv1(input)
        input = self.Conv2(input)       #view can be understood as resize
        #input.size() = [100, 32, 7, 7], 100 is quantity per batch, 32 is thickness, picture size is 7*7
        #When a dimension is -1, its size is automatically calculated (the principle is that the total amount of data is constant):
        input = input.view(input.size(0), -1) #(batch=100, 1568), the end result is to compress a two-dimensional picture into one dimension (the amount of data remains constant)
        #Finally, connect to a full connection layer with output of 10:[100,1568]*[1568,10]=[100,10]
        output = self.Linear(input)
        return output






#Read Network Framework
cnn = CNN()
#Read Weights:
cnn.load_state_dict(torch.load('EMNIST_CNN.pkl'))


#test_x:(10000 rows and 1 column, each column element is 28*28 matrix) 
# Provide your own data for testing:
my_img = plt.imread("Emnist_letters_png/My_jpg/g.jpg")
my_img = my_img[:,:,0] #Convert to Single Channel
my_img = cv2.resize(my_img,(28,28))#Convert to 28*28 size
my_img = torch.from_numpy(my_img)#Convert to Tensor
my_img = torch.unsqueeze(my_img, dim = 0)#Add a dimension
my_img = torch.unsqueeze(my_img, dim = 0)/255. #Add another dimension and map the gray scale between (0,1)       
#print(my_img.size())#torch.Size([1, 1, 28, 28]) Convolution layer requires four dimensions of input






#Visualization section:

#Input original image:
plt.imshow(my_img.squeeze())
plt.show()



#Conv1:
cnt = 1
my_img = cnn.Conv1(my_img)
img = my_img.squeeze()
for i in img.squeeze():

    plt.axis('off')
    fig = plt.gcf()
    fig.set_size_inches(5,5)#Output width*height pixels
    plt.margins(0,0)

    plt.imshow(i.detach().numpy())
    plt.subplot(4, 4, cnt)
    plt.axis('off')
    plt.imshow(i.detach().numpy())
    cnt += 1
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
plt.show()



#Conv2:
cnt = 1
my_img = cnn.Conv2(my_img)
img = my_img.squeeze()
for i in img.squeeze():

    plt.axis('off')
    fig = plt.gcf()
    fig.set_size_inches(5,5)#Output width*height pixels
    plt.margins(0,0)

    plt.imshow(i.detach().numpy())
    plt.subplot(4, 8, cnt)
    plt.axis('off')
    plt.imshow(i.detach().numpy())
    cnt += 1
#plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
plt.show()




#Full Connection Layer:
my_img = my_img.view(my_img.size(0), -1)
fig = plt.gcf()
fig.set_size_inches(10000,4)#Output width*height pixels
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
plt.margins(0,0)


my_img = cnn.Linear[0](my_img)
plt.subplot(3, 1, 1)
plt.imshow(my_img.detach().numpy())

my_img = cnn.Linear[1](my_img)
my_img = cnn.Linear[2](my_img)
my_img = cnn.Linear[3](my_img)
plt.subplot(3, 1, 2)
plt.imshow(my_img.detach().numpy())

my_img = cnn.Linear[4](my_img)
my_img = cnn.Linear[5](my_img)
plt.subplot(3, 1, 3)
plt.imshow(my_img.detach().numpy())

plt.show()



#Output prediction results:
pred_y = int(torch.max(my_img,1)[1])
#chr() converts a number to the corresponding ASCII character
print('\npredict character: %c or %c' % (chr(pred_y+65),chr(pred_y+97)))

FileNotFoundError: [Errno 2] No such file or directory: 'EMNIST_CNN.pkl'